In [1]:
import json
import os
import requests

from typing import Any

customer_id = "2906470669"
corpus_id = "2"
api_key = os.getenv("VECTARA_API_KEY") or "VECTARA_API_KEY"
url = f"https://api.vectara.io/v1/upload?c={customer_id}&o={corpus_id}"

post_headers = {
    "x-api-key": api_key,
    "customer-id": customer_id
}

In [2]:
directory = "./qna"
qna_files = []
for root, dirs, files in os.walk(directory):
    for file_name in files:
        # Check if the file is a markdown file
        if file_name.endswith('.md'):
            file_path = os.path.join(root, file_name)

            # Read original md file content
            md_file_path = file_path.replace("/qna/", "/markdown/").replace("_qna.md", ".md")
            with open(md_file_path, "r", encoding="utf-8") as file:
                file_lines = file.readlines()
                file_name_stubs = file_name.split(".")[0].split("_")
                meta_data = dict(
                    title=file_lines[0][1:].strip(),
                    url="",
                    category=file_lines[1][12:].strip(),
                    category_slug=file_name_stubs[0],
                    slug=file_name_stubs[1],
                    file_name=file_name,
                    file_path=file_path,
                    qna=True
                )
                # Append the file path and content to the markdown_files list
                qna_files.append((file_path, meta_data))

In [3]:
qna_files[0]

('./qna/TransactionDetails_PurchasingaPortionoftheCompany_qna.md',
 {'title': 'Purchasing a Portion of the Company',
  'url': '',
  'category': 'Transaction Details',
  'category_slug': 'TransactionDetails',
  'slug': 'PurchasingaPortionoftheCompany',
  'file_name': 'TransactionDetails_PurchasingaPortionoftheCompany_qna.md',
  'file_path': './qna/TransactionDetails_PurchasingaPortionoftheCompany_qna.md',
  'qna': True})

In [4]:
responses = []
for qnaf in qna_files:
    files: Any = {
        "file": (qnaf[1]["file_name"], open(qnaf[0], "rb")),
        "doc_metadata": (None, json.dumps(qnaf[1])),
    }
    response = requests.post(url, files=files, verify=True, headers=post_headers)
    responses.append(response)

In [5]:
errors = [r for r in responses if r.status_code != 200]

In [6]:
errors

[]

In [7]:
responses[0].text

'{"response":{\n  "status": {\n  },\n  "quotaConsumed": {\n    "numChars": "10720",\n    "numMetadataChars": "2156"\n  }\n}}'